# Food in Art

In [48]:
import pandas as pd
import requests
from urllib.parse import unquote, urlparse


In [49]:

df = pd.read_csv('data/wikidata_paintings_final_with_wiki_url.csv')
#df = paintings[paintings['wikipedia_url'].notnull()]

In [50]:
def retrieve_wikipedia_article_text(df):
    session = requests.Session()
    def get_article_text(url):
        if pd.isnull(url):
            return None
        try:
            # Parse the URL to extract the article title
            path = urlparse(url).path  # e.g., '/wiki/Article_Title'
            parts = path.split('/')
            if len(parts) < 3 or parts[1] != 'wiki':
                return None  # Invalid URL format
            title = parts[2]
            # Decode any percent-encoded characters
            title = unquote(title)
            # Prepare the Wikipedia API request
            api_url = 'https://en.wikipedia.org/w/api.php'
            params = {
                'action': 'query',
                'format': 'json',
                'prop': 'extracts',
                'explaintext': True,
                'redirects': True,
                'titles': title
            }
            # Make the API request
            response = session.get(api_url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            # Extract the page content
            pages = data.get('query', {}).get('pages', {})
            page = next(iter(pages.values()))
            extract = page.get('extract', None)
            print(f"Retrieved article for URL {url}")
            return extract
        except Exception as e:
            print(f"Error retrieving article for URL {url}: {e}")
            return None
    # Apply the function to the DataFrame
    df['article_text'] = df['wikipedia_url'].apply(get_article_text)
    return df

In [ ]:

paintings_with_articles = retrieve_wikipedia_article_text(df)
# Step 7: Optional - Save to CSV
paintings_with_articles.to_csv('data/wikidata_paintings_final_with_wiki_articles.csv', index=False)

print("Article retrieval complete.")

In [ ]:
df

In [ ]:
paintings_with_articles